In [2]:
import torch
import numpy as np
import pandas as pd 
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn import metrics
import seaborn as sns
import torch.nn.functional as F

In [4]:
class ConvolutionNN(nn.Module):

    def __init__(self, loss_method = "BCE", opt_method = "SGD", lr = 0.01, threshold = 0.3, class_weights = None, epochs=5, input_dimension = None, dropout_list=None, linear_act = None, flattened_size = None, linear_layer = None, layer_list = None,default_layer=None, pool_counts=None, activation_count=None, pool=None, activations=None, layer_count = None, dropout_count = None, dropout_rate= None, layer_default = 30, output_dimension= None, kernel = 3, padding=1):
        super().__init__()
        
        self.input_dimension = input_dimension
        self.output_dimension = output_dimension
        self.kernel = kernel
        self.padding = padding
        
        self.dropout_count = dropout_count
        self.dropout_rate = dropout_rate
        self.layer_list = layer_list
        self.dropout_rate = dropout_rate
        self.dropout_list = dropout_list
        self.dropout_count = dropout_count
        self.pool_counts = pool_counts
        self.pool = pool
        self.layer_count = layer_count
        self.activations = activations
        self.activation_count = activation_count
        self.layer_default = layer_default
        self.default_layer = default_layer
        self.linear_layer = linear_layer
        self.flattened_size = flattened_size
        self.loss_method = loss_method
        self.opt_method = opt_method
        self.lr = lr 
        self.class_weights = class_weights
        self.epochs = epochs
        self.threshold = threshold
        
        if self.class_weights is not None:
            self.class_weights = torch.tensor([class_weights[1], class_weights[0]], dtype= torch.float32)
        else: 
            self.class_weights = class_weights
            
        layer_dict = {}
        
        if self.layer_list is None: 
            
            layer = [self.input_dimension] + [self.default_layer]*self.layer_count + [self.output_dimension]

        else: 
            
            layer = self.layer_list

        for i in range(1, len(layer)):

            layer_dict[f"conv{i}"] = nn.Conv1d(layer[i-1], layer[i], kernel_size = self.kernel, padding = self.padding)

            act = None
            if self.activations is None:
                
                layer_dict[f"act{i-1}"] = nn.ReLU()

            else: 

                if len(self.activations) < (len(layer)):
                    need_act = (len(layer)) - len(self.activations)
                    act = self.activations + ["identity"]*need_act 
                    
                layer_dict[f"act{i-1}"] = self.get_activation(act[i-1])

            pooling = None            
            if self.pool is None:
                
                layer_dict[f"pooling{i-1}"] = nn.MaxPool1d(2,2)

            else: 
                
                if len(self.pool) < (len(layer)):
                    need_pool = (len(layer)) - len(self.pool)
                    pooling = self.pool + [2]*need_pool
                layer_dict[f"pooling{i-1}"] = nn.MaxPool1d(pooling[i-1], pooling[i])

            drop = None
            if self.dropout_list is None: 
                
                layer_dict[f"dropout{i-1}"] = nn.Dropout(self.dropout_rate)

            else: 
                if len(self.dropout_list) < (len(layer)):
                    need_drop = (len(layer)) - len(self.dropout_list)
                    drop = self.dropout_list + [0.3]*need_drop 
                layer_dict[f"dropout{i-1}"] = nn.Dropout(drop[i-1])

    
        
        self.linear1 = nn.Linear(self.flattened_size, self.linear_layer)
        self.linear2 = nn.Linear(self.linear_layer, self.output_dimension)

        self.process = nn.ModuleDict(layer_dict)
        print(self.process)
    

    def forward(self, x):
        
        for m in self.process.values():
            x = m(x)

        x = torch.flatten(x, 1)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        
        print(x)
        return x
        
    def get_activation(self, activation_):

        if activation_ == "relu": 
            return nn.ReLU()

        elif activation_ == "tanh": 
            return nn.Tanh()

        elif activation_ == "identity": 
            return nn.Identity()

    def get_loss(self):
        
        
        if self.loss_method == "BCE":
            return nn.BCELoss()
        
        elif self.loss_method == "L1":
            return nn.L1Loss()
        
        elif self.loss_method == "MSE":
            return nn.MSELoss()

        elif self.loss_method == "CE":
            return nn.CrossEntropyLoss(weight = self.class_weights)

        elif self.loss_method == "BCEwLogit":
            if self.class_weights is not None:
                pos_weight = torch.tensor([self.class_weights[1] / self.class_weights[0]])
                return nn.BCEWithLogitsLoss(pos_weight = pos_weight)
            else: 
                return nn.BCEWithLogitsLoss()

        else:
            raise ValueError(f"{self.loss_method} is not valid!")

    def get_optimizer(self):
        
        if self.opt_method == "SGD":
            return torch.optim.SGD(params = self.parameters(), lr = self.lr)

        elif self.opt_method == "Adam":
            return torch.optim.Adam(params = self.parameters(), lr = self.lr)

        elif self.opt_method == "RMSprop":
            return torch.optim.RMSprop(params = self.parameters(), lr = self.lr)

        else: 
            raise ValueError(f"{self.opt_method} is not valid!")

    def train_model(self, train_loader, optimizer=None, loss_fn=None):#https://www.geeksforgeeks.org/how-to-implement-neural-networks-in-pytorch/

        loss_fn = self.get_loss()
        optimizer = self.get_optimizer()        
        size = len(train_loader.dataset)
        t_loss=[]
        for e in range(self.epochs):
            self.train()
            train_loss = 0
            for batch, (X, y) in enumerate(train_loader):
                X = X.unsqueeze(1)
                y_logits = self(X).squeeze()
                
                loss = loss_fn(y_logits, y)
    
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
    
                train_loss += loss.item() * X.size(0)
    
            train_loss_ = train_loss/len(train_loader.dataset) 
            t_loss.append(train_loss_)
        return t_loss

    def test_model(self, test_loader, loss_fn=None):
        loss_fn = self.get_loss()
        self.eval()
        
        
        with torch.inference_mode():
            val_loss = []
            for e in range(self.epochs):
                test_loss = 0
                for X, y in test_loader: 
                    X = X.unsqueeze(1)
                    y_logits = self(X).squeeze()
                    test_loss += loss_fn(y_logits, y).item() * y.size(0)
                    
                test_loss_ = test_loss/len(test_loader.dataset)
                val_loss.append(test_loss_)
            return val_loss

    def analysis(self, l1, l2):
        #https://www.geeksforgeeks.org/how-to-create-a-multiline-plot-using-seaborn/
        l1_arr = np.array(l1)
        l2_arr = np.array(l2)

        epoch_l = [i for i in range(1, self.epochs + 1)]
        epoch_arr = np.array(epoch_l)

        sns.lineplot(x = epoch_arr, y = np.log(l1_arr), label="Train Error")
        sns.lineplot(x = epoch_arr, y = np.log(l2_arr), label="Valid Error")
        plt.legend()
        plt.show()


    def predict(self, test):
    #https://discuss.pytorch.org/t/how-to-use-pytorch-to-output-the-probability-of-binary-classfication/101043/2
    #https://codesignal.com/learn/courses/building-a-neural-network-in-pytorch/lessons/making-predictions-with-a-trained-pytorch-model
    #https://discuss.pytorch.org/t/how-to-make-pytorch-model-predict/167950
        predictions=[]
        labels = []
        probs=[]
        if self.loss_method == "BCE": 
            
            self.eval()
            with torch.inference_mode():
                for X, y in test:
                    X = X.unsqueeze(1)
                    output = self(X).squeeze()
                    preds = (output > self.threshold).int()
                    predictions.append(preds)
                    labels.append(y)
                    probs.append(output)
                    
        elif self.loss_method == "CE": 
            self.eval()
            with torch.inference_mode():
                for X, y in test:
                    X = X.unsqueeze(1)
                    output = self(X)
                    prob = torch.argmax(output, dim=1)
                    preds = (prob>self.threshold).int()
                    predictions.append(preds)
                    labels.append(y)
                    probs.append(prob)
                    
        else:
            self.eval()
            with torch.inference_mode():
                for X, y in test:
                    X = X.unsqueeze(1)
                    output = self(X).squeeze()
                    prob = torch.sigmoid(output)
                    preds = (prob > self.threshold).int()
                    predictions.append(preds)
                    labels.append(y)
                    probs.append(prob)

        all_preds = torch.cat(predictions).ravel()
        all_labels = torch.cat(labels).ravel()
        all_probs = torch.cat(probs).ravel()
        
        return all_probs, all_preds, all_labels
        
    def report(self, test, pred, labels): 

        all_probs, all_preds, all_labels = self.predict(test)
        all_probs_arr = all_probs.detach().numpy().ravel()
        all_preds_arr = all_preds.detach().numpy().ravel()
        all_labels_arr = all_labels.detach().numpy().ravel()
        
        cm = confusion_matrix(all_labels_arr, all_preds_arr)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt="d")
        plt.title("Confusion Matrix")
        plt.ylabel("Actual Class")
        plt.xlabel("Predicted Class")

        plt.show()
            